In [2]:
#prepei na einai egkatesthmena
#pip install gurobipy
#pip install numpy

In [3]:
from gurobipy import *
import numpy as np

#                    Σταύρος Βασίλειος Μπουλιόπουλος 9671 
#                  Πρώτο πρόβλημα Επιχειρησιακής Έρευνας
Πρώτο πρόβλημα Επιχειρησιακής Έρευνας για το εργοστάσιο παραγαγωγής χρωμάτων και τον διαδοχικό καθαρισμό χρωμάτων από τις μηχανές για την παραγωγή νέου.
## Problem definition
Έχω αποχρώσεις 1 εώς N=5. Χρώματα-colours $C = {1...N}$. Περίοδο παραγωγής για κάθε χρώμα i είναι $T_i$. Χρόνος καθαρισμού ετοιμασίας από χρώμα i σε j είναι $t_{ij}$. Μεταβλητή binary απόφασης $x_{ij}$ που ισούται με 1 αν πάρουμε χρώμα j μετά από το i.
## Objective function
Θέλω να ελαχιστοποιήσω τον χρόνο παραγωγής και καθαρισμού ,δηλαδή 
$$min \sum_{i\in c} \sum_{j\in c,j\ne i} (T_i+t_{ij})\cdot x_{ij} = total$$

## Constraints-s.t.
Εξηγούνται με σχόλια greeklish στον κώδικα.

In [4]:
#affiliate pinakas sysxetishs 
t = np.array([[0, 11, 7, 13, 11],
             [5, 0, 13, 15, 15],
             [13, 15, 0, 23, 11],
             [9, 13, 5, 0, 3],
             [3, 7, 7, 7, 0]], np.int32)
#fixed kostos to opoio den mas ephreazei sto optimization
T = [40, 35, 45, 32, 50]
mdl = Model('prob1')
#mdl.setParam('OutputFlag', False)
n = len(T)  #plh8os apoxrwsewn
x = mdl.addMVar((t.shape[0],t.shape[1]), vtype = GRB.BINARY, name = "x")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-03


In [5]:
#periorismoi s.t.
for i in range(n):
    #h diagwnios mou isoutai me mhden giati den mporw na exw diadoxh kai ka8arismo tou idiou xrwmatos
    mdl.addConstr(x[i,i] == 0)
    #to epomeno xrwma pros xrwmatismo einai mono 1
    mdl.addConstr(sum(x[i,:]) == 1)
    #den mporw na paragw 2 xrwmata thn idia xronikh stigmh
    mdl.addConstr(sum(x[:,i]) == 1)

#gia na apofygw anakyklwseis
for i in range(n):
    for j in range(n):
        mdl.addConstr((x[i,j]+x[j,i]) <= 1)



obj_fn = sum(x[i,j]*t[i,j] for i in range(n) for j in range(n))
mdl.setObjective(obj_fn, GRB.MINIMIZE)

mdl.setParam('OutputFlag', False)
mdl.optimize()

X = np.zeros(n*n)
ct = 0
for v in mdl.getVars():
    X[ct] = int(v.x)
    ct += 1

In [7]:
X = np.reshape(X, (-1, 5))
X = np.transpose(X)
colourOrder = np.zeros(n+1)
colourOrder[0] = 0; #1o xrwma 
stayWhile = True
colourNow = 0
currentIdx = 1
while(stayWhile):
    colourDiadoxh = -1; 
    for j in range(n):
        if (X[colourNow,j] == 1):
            colourDiadoxh = j 

    
    colourOrder[currentIdx] = colourDiadoxh
    colourNow = colourDiadoxh
    currentIdx += 1
    if (currentIdx > n):
        stayWhile = False 

print("gia beltisto ka8arismo apo")
print(int(colourOrder[0])+1)
totalColouring = sum(T)
totalClean = 0
for i in range(1,n+1):
    
    totalClean += t[int(colourOrder[i-1]),int(colourOrder[i])]
    print("paw se")
    print(int(colourOrder[i])+1)
    
total = totalColouring + totalClean
print("ekleise o kyklos, apo epomenh ebdomada pali")
print(totalClean)
print("(standard xroniko kostos paragwghs 5 xrwmatwn={0} ) + (beltisto kostos \ndiadoxikou ka8arismou={1} ) = synoliko kostos {2} lepta".format(totalColouring, totalClean, total))

gia beltisto ka8arismo apo
1
paw se
4
paw se
3
paw se
5
paw se
2
paw se
1
ekleise o kyklos, apo epomenh ebdomada pali
41
(standard xroniko kostos paragwghs 5 xrwmatwn=202 ) + (beltisto kostos 
diadoxikou ka8arismou=41 ) = synoliko kostos 243 lepta
